# Wikipedia pageviews Bronze
Executes by run_date as string formatted 'yyyy-mm-dd
Assumes s3 folder structure: "s3a://data/wikipedia_pageviews/YYYY/YYYY-MM/DD/pageviews-YYYYMMDD-hhmmss.gz"
Performs: 
* ingestion
* column naming
* column casting
* get the date from the filename
* partitioning
* storing as delta table

Note: there will be a bunch of warnings and some exceptions from the Hive catalog, that's because it doesn't support schema updates. Everything works just fine anyway.
In databricks the Unity Catalog is used, but it's proproetary so we'll have to make do with Hive Catalog.

In [ ]:
import datetime as dt

execution_date = "2025-05-15T01:00:00+00:00"
execution_id = f"wikipedia_pageviews_bronze-{dt.datetime.now():%Y%m%d-%H0000}"
full_refresh = False
bronze_db = "bronze"
root_prefix = "wikipedia_pageviews"
data_bucket = "data"

## Bronze pipeline

In [ ]:
import os

from freeds.s3 import as_urls, list_files, list_files_for_dates
from freeds.spark import get_spark_session, show_spark_info
from freeds.utils import date_range
from pyspark.sql.functions import col, input_file_name, substring, to_date
from pyspark.sql.types import IntegerType, StringType, StructField, StructType

# s3_path = "s3a://data/wikipedia_pageviews/2025/2025-03/*/*.gz"
# s3_path = "s3a://data/wikipedia_pageviews/2025/2025-03/11/pageviews-20250311-220000.gz"
# s3_path = "s3a://data/test.csv"


if not os.environ.get("SPARK_CONF_DIR"):
    os.environ["SPARK_CONF_DIR"] = "/opt/freeds/spark/conf"

print("Getting a spark session")
spark = get_spark_session(execution_id)
show_spark_info(spark)

print(f"Creating the {bronze_db} database")
spark.sql(f"CREATE DATABASE IF NOT EXISTS {bronze_db}")

schema = StructType(
    [
        StructField(name="domain_code", dataType=StringType(), nullable=True),
        StructField("page_title", StringType(), True),
        StructField("count_views", StringType(), True),
    ]
)
if full_refresh:
    file_spec = list_files(prefix=root_prefix, bucket_name=data_bucket)
else:
    run_dates = date_range(execution_date=execution_date, length=2)
    file_spec = list_files_for_dates(dates=run_dates, root_prefix=root_prefix, bucket_name=data_bucket)

file_spec = as_urls(file_spec, bucket_name=data_bucket)

print(f"Loading data, found {len(file_spec)} files for {'full refresh' if full_refresh else run_dates}.")

# this only helps if part of the files are missing
spark.conf.set("spark.sql.files.ignoreMissingFiles", "true")
# in our case all files were missing, wrap in try/except
reader = (
    spark.read.format("csv")
    .option("delimiter", " ")
    .option("header", "false")
    .option("inferSchema", "false")
    .schema(schema)
)

data = reader.load(file_spec)
print(f"Files to load in spark:{len(data.inputFiles())}, listing first 10:")
for f in data.inputFiles()[:10]:
    print(f)

data_enriched = (
    data.na.drop(subset=["domain_code"])
    .filter(~col("page_title").contains(":"))
    .filter(~col("page_title").isin("-", "Main_Page", "Forside", "Hauptseite", "wiki.phtml"))
    .withColumn("country_code", substring(col("domain_code"), 1, 2))
    .filter(col("domain_code").isin("sv", "dk", "no", "de", "en"))
    .withColumn("count_views", col("count_views").cast(IntegerType()))
    .withColumn("file_name", input_file_name())
    .withColumn("date", to_date(substring(col("file_name"), -18, 8), "yyyyMMdd"))
)

table_name = f"{bronze_db}.wikipedia_page_reads"
# silence a few bulky hive catalog warnings
spark.sparkContext.setLogLevel("ERROR")
if not full_refresh:
    spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
(
    data_enriched.write.mode("overwrite")  # Options: 'overwrite', 'append', 'ignore', 'error' (default)
    .option("mergeSchema", "true")
    .format("delta")  # Options: 'parquet', 'csv', 'json', 'orc', etc.
    .partitionBy("date")
    .saveAsTable(table_name)
)
print(f"Updated delta table: {table_name}")
spark.stop()
print(f"All done: {execution_id}")